In [11]:
!pip install numpy matplotlib scikit-learn pillow


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
# Importando bibliotecas necessárias
import numpy as np  # Biblioteca para cálculos matemáticos
import matplotlib.pyplot as plt  # Biblioteca para visualização de gráficos
from sklearn.model_selection import train_test_split  # Função para dividir os dados em treino e teste
from sklearn.preprocessing import LabelBinarizer  # Função para binarizar rótulos (Cat/Dog)
from PIL import Image  # Biblioteca para carregar e processar imagens
import os  # Biblioteca para interagir com o sistema de arquivos


In [13]:
# Função para carregar o dataset com base nos diretórios 'Cat' e 'Dog'
def load_data(path):
    categories = ['Cat', 'Dog']  # As duas classes do dataset
    file_paths = []  # Lista para armazenar os caminhos das imagens
    labels = []  # Lista para armazenar os rótulos (0 para Cat, 1 para Dog)

    # Iterando sobre as categorias
    for category in categories:
        folder = os.path.join(path, category)  # Caminho para a pasta 'Cat' ou 'Dog'
        label = categories.index(category)  # Atribui 0 a 'Cat' e 1 a 'Dog'

        # Itera sobre todos os arquivos da pasta
        for filename in os.listdir(folder):
            file_path = os.path.join(folder, filename)  # Caminho completo da imagem
            file_paths.append(file_path)  # Adiciona o caminho da imagem à lista
            labels.append(label)  # Adiciona o rótulo correspondente

    return np.array(file_paths), np.array(labels)  # Retorna os caminhos e rótulos como arrays

In [5]:
# Função para pré-processar cada imagem
def preprocess_image(image_path, target_size=(64, 64)):
    img = Image.open(image_path).convert('L')  # Converte a imagem para escala de cinza
    img = img.resize(target_size)  # Redimensiona a imagem para 64x64 pixels
    img_array = np.array(img) / 255.0  # Normaliza os valores dos pixels (0 a 1)
    return img_array.flatten()  # Retorna a imagem como um vetor unidimensional

# Carregar o dataset de gatos e cães
dataset_path = os.path.abspath('D:/Codes/cnn/dataset')  # Diretório contendo as pastas 'Cat' e 'Dog'
file_paths, labels = load_data(dataset_path)  # Carrega as imagens e rótulos

# Pré-processar todas as imagens (converter em arrays numéricos)
X = np.array([preprocess_image(img_path) for img_path in file_paths])
y = labels.reshape(-1, 1)  # Os rótulos (0 ou 1) são armazenados como uma matriz de uma coluna

# Dividir o dataset em conjunto de treino (80%) e teste (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Função para inicializar os parâmetros da rede neural (pesos e bias)
def initialize_parameters(n_x, n_h, n_y):
    np.random.seed(1)  # Garante a reprodutibilidade dos resultados
    W1 = np.random.randn(n_h, n_x) * 0.01  # Inicializa os pesos da camada oculta com valores aleatórios
    b1 = np.zeros((n_h, 1))  # Inicializa o bias da camada oculta como zeros
    W2 = np.random.randn(n_y, n_h) * 0.01  # Inicializa os pesos da camada de saída
    b2 = np.zeros((n_y, 1))  # Inicializa o bias da camada de saída como zeros
    
    # Armazena os parâmetros em um dicionário
    parameters = {"W1": W1, "b1": b1, "W2": W2, "b2": b2}
    return parameters  # Retorna os parâmetros

# Definindo os tamanhos das camadas
n_x = X_train.shape[1]  # Número de features (tamanho da imagem vetorizada)
n_h = 4  # Neurônios na camada oculta (valor arbitrário)
n_y = 1  # Número de saídas (1 para classificação binária)

# Inicializa os parâmetros da rede
parameters = initialize_parameters(n_x, n_h, n_y)

In [7]:
# Função de forward propagation (propagação para frente)
def forward_propagation(X, parameters):
    W1 = parameters['W1']  # Pesos da camada oculta
    b1 = parameters['b1']  # Bias da camada oculta
    W2 = parameters['W2']  # Pesos da camada de saída
    b2 = parameters['b2']  # Bias da camada de saída
    
    # Cálculo do Z1 (entrada da função de ativação na camada oculta)
    Z1 = np.dot(W1, X.T) + b1
    A1 = np.tanh(Z1)  # Função de ativação (tangente hiperbólica) na camada oculta
    Z2 = np.dot(W2, A1) + b2  # Cálculo do Z2 (entrada da função de ativação na camada de saída)
    A2 = 1 / (1 + np.exp(-Z2))  # Função sigmoide na camada de saída (probabilidade entre 0 e 1)
    
    # Armazena os valores intermediários no cache para uso na backpropagation
    cache = {"Z1": Z1, "A1": A1, "Z2": Z2, "A2": A2}
    return A2, cache  # Retorna a saída final e os valores intermediários


In [8]:
# Função para calcular o custo (erro) da predição
def compute_cost(A2, Y, parameters):
    m = Y.shape[0]  # Número de exemplos no dataset
    # Cálculo da função de custo (cross-entropy)
    logprobs = Y.T * np.log(A2) + (1 - Y.T) * np.log(1 - A2)
    cost = -np.sum(logprobs) / m  # Calcula a média do erro
    cost = np.squeeze(cost)  # Garante que o custo seja um valor escalar
    return cost  # Retorna o custo


In [9]:
# Função para realizar a backward propagation (retropropagação)
def backward_propagation(parameters, cache, X, Y):
    m = X.shape[0]  # Número de exemplos
    
    # Recupera os parâmetros
    W1 = parameters['W1']
    W2 = parameters['W2']
    
    # Recupera os valores intermediários do cache
    A1 = cache['A1']
    A2 = cache['A2']
    
    # Calcula o gradiente da função de custo em relação a Z2
    dZ2 = A2 - Y.T
    dW2 = (1 / m) * np.dot(dZ2, A1.T)  # Gradiente dos pesos W2
    db2 = (1 / m) * np.sum(dZ2, axis=1, keepdims=True)  # Gradiente do bias b2
    
    # Calcula o gradiente da função de custo em relação a Z1
    dZ1 = np.dot(W2.T, dZ2) * (1 - np.power(A1, 2))  # Derivada da função tangente hiperbólica
    dW1 = (1 / m) * np.dot(dZ1, X)  # Gradiente dos pesos W1
    db1 = (1 / m) * np.sum(dZ1, axis=1, keepdims=True)  # Gradiente do bias b1
    
    # Armazena os gradientes em um dicionário
    grads = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}
    return grads  # Retorna os gradientes


In [10]:
# Função para atualizar os parâmetros usando o gradient descent
def update_parameters(parameters, grads, learning_rate=0.01):
    W1 = parameters['W1'] - learning_rate * grads['dW1']  # Atualiza W1
    b1 = parameters['b1'] - learning_rate * grads['db1']  # Atualiza b1
    W2 = parameters['W2'] - learning_rate * grads['dW2']  # Atualiza W2
    b2 = parameters['b2'] - learning_rate * grads['db2']  # Atualiza b2
    
    # Armazena os parâmetros atualizados
    parameters = {"W1": W1, "b1": b1, "W2": W2, "b2": b2}
    return parameters  # Retorna os parâmetros atualizados


In [11]:
# Função que realiza o treinamento da rede neural
def train_model(X, Y, n_h, num_iterations=10000, learning_rate=0.1):
    n_x = X.shape[1]  # Tamanho da camada de entrada
    n_y = 1  # Número de unidades de saída (classificação binária)
    
    # Inicializa os parâmetros
    parameters = initialize_parameters(n_x, n_h, n_y)
    
    # Itera sobre o número de iterações desejado
    for i in range(num_iterations):
        A2, cache = forward_propagation(X, parameters)  # Executa a forward propagation
        cost = compute_cost(A2, Y, parameters)  # Calcula o custo
        grads = backward_propagation(parameters, cache, X, Y)  # Executa a backward propagation
        parameters = update_parameters(parameters, grads, learning_rate)  # Atualiza os parâmetros
        
        # A cada 100 iterações, imprime o custo
        if i % 100 == 0:
            print(f"Custo após a iteração {i}: {cost}")
    
    return parameters  # Retorna os parâmetros treinados

# Treina a rede neural com o conjunto de treinamento
parameters = train_model(X_train, y_train, n_h=4, num_iterations=1000, learning_rate=0.01)

Custo após a iteração 0: 0.6930949692375822
Custo após a iteração 100: 0.6929159155264814
Custo após a iteração 200: 0.6926644104598834
Custo após a iteração 300: 0.6921732342999208
Custo após a iteração 400: 0.6909937002906821
Custo após a iteração 500: 0.688202832561125
Custo após a iteração 600: 0.6823529851744597
Custo após a iteração 700: 0.6717142002367913
Custo após a iteração 800: 0.6548166460626551
Custo após a iteração 900: 0.6301114718387674


In [12]:
# Função para realizar a predição usando os parâmetros treinados
def predict(X, parameters):
    A2, _ = forward_propagation(X, parameters)  # Executa a forward propagation
    predictions = (A2 > 0.5).astype(int)  # Classifica como 1 (Dog) se A2 > 0.5, senão 0 (Cat)
    return predictions.T  # Retorna as predições

# Realiza a predição no conjunto de teste
y_pred = predict(X_test, parameters)

# Calcula a precisão do modelo
accuracy = np.mean(y_pred == y_test.T) * 100
print(f"Precisão do modelo no conjunto de teste: {accuracy:.2f}%")

Precisão do modelo no conjunto de teste: 49.55%
